# Молекулярная динамика

Экспериментальные исследования не позволяют проследить за динамикой молекулярной системы на атомарном уровне. С этой точки зрения моделирование выглядит многообещающе.

![]() <img src="img/myoglobin_licorice2.png"  width="400">

Гипотезы:

1. Молекулярную систему можно представить в виде набора взаимодействующих центров, а их движение описать законами классической механики.

2. Гипотеза аддитивности: общая энергия взаимодействия — сумма отдельных межчастичных взаимодействий.

3. Гипотеза переносимости: параметризация взаимодействий может опираться на ограниченный набор параметров.


## Силовое поле

Потенциальная функция (силовое поле) молекулярной динамики:

$$
V = V_{bonded} + V_{nb}
$$

Валентные взаимодействия (количество слагаемых ~ количество атомов):

$$
\begin{split}
V_{bonded} = &\sum_{(i,j)\in bonds}\frac{k^s_{ij}}{2}(r_{ij}-r_{ij}^0)^2 + \\
             &\sum_{(i,j,k)\in angles}\frac{k^{\theta}_{ijk}}{2}(\theta_{ijk}-\theta_{ijk}^0)^2 + \\
             &\sum_{(i,j,k,l)\in dihedrals}k^{\phi}_{ijkl}(1-\cos(n\phi_{ijkl} - \phi_{ijkl}^0)) + \\
             &\sum_{(i,j,k,l)\in impropers}k^{\psi}_{ijkl}(\psi_{ijkl}-\psi_{ijkl}^0)^2
\end{split}
$$

Невалентные взаимодействия (каждый с каждым):

$$
V_{nb} = \sum_{i,j}\varepsilon_{ij}\left[\left(\frac{\sigma_{ij}}{r_{ij}}\right)^{12}-2\left(\frac{\sigma_{ij}}{r_{ij}}\right)^{6}\right]
$$

Уравнения движения:

$$
\begin{split}
m_i\frac{d\mathbf{V}_{i}}{dt} &= \mathbf{f}_{i}\\
\frac{d\mathbf{r}_i}{dt} &= \mathbf{v}_{i}
\end{split}
$$

, где:

$$
\mathbf{f}_i = \nabla_i V = \begin{bmatrix}\frac{\partial}{\partial x_i}\\\frac{\partial}{\partial y_i}\\\frac{\partial}{\partial z_i}\end{bmatrix}V
$$



In [1]:
!gmx --version

                         :-) GROMACS - gmx, 2022.5 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/zhmurov/git/artemzhmurov/5cb-tutorial
Command line:
  gmx --version

GROMACS version:    2022.5
Precision:          mixed
Memory model:       64 bit
MPI library:        thread_mpi
OpenMP support:     enabled (GMX_OPENMP_MAX_THREADS = 128)
GPU support:        disabled
SIMD instructions:  AVX2_256
CPU FFT library:    fftw-3.3.8-sse2-avx
GPU FFT library:    none
RDTSCP usage:       enabled
TNG support:        enabled
Hwloc support:      disabled
Tracing support:    disabled
C compiler:         /usr/bin/cc GNU 11.2.0
C compiler flags:   -mavx2 -mfma -Wno-missing-field-initializers -fexcess-precision=fast -funroll-all-loops -O3 -DNDEBUG
C++ compiler:       /usr/bin/c++ GNU 11.2.0
C++ compiler flags: -mavx2 -mfma -Wno-missing-field-initializers -fexcess-precision=fast -funroll-all-loops -fopenmp -O3 -DNDEBUG



Следующая ячейка создает отдельную директорию и переходит в нее. Обратите внимание, что перед командой 'cd' стоит знак '%', а не '!' для того, чтобы остаться в новой папке далее, при выполнении последующих ячеек.

In [2]:
!mkdir sim
%cd sim

/home/zhmurov/git/artemzhmurov/5cb-tutorial/sim


## Создание файлов координат и топологии

Для описания взаимодействий между атомами в молекулярной динамике используется силовое поле. Силовое поле представляет собой набор файлов, в которых перечисленны возможные типы атомов, то как они связаны между собой в ряде молекул, какие параметры ковалентных взаимодействий у этих связей, какие у атомов заряды и т.д. Выбор силового поля является очень важным этапом в молекулярном моделировании. Например, силовые поля разработанные для биологических молекул, зачастую работают корректно только в физиологическом диапазоне температур и давлений.

Здесь мы будем использовать модифицированное силовое поле TRaPPE-UA, которое изначально разрабатывалось с целью корректно описывать фазовые состояния молекул. Поле было адаптировано к использованию в пакете GROMACS, но не входит в его стандартную поставку. Для того, чтобы скачать силовое поле [здесь](https://github.com/zhmurov/trappeua/archive/refs/heads/master.zip):

In [3]:
%%bash

wget https://github.com/zhmurov/trappeua/archive/refs/heads/master.zip
unzip master.zip
rm master.zip
mv trappeua-master/trappeua.ff .
rm -rf trappeua-master

--2023-11-20 14:17:35--  https://github.com/zhmurov/trappeua/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/zhmurov/trappeua/zip/refs/heads/master [following]
--2023-11-20 14:17:36--  https://codeload.github.com/zhmurov/trappeua/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

     0K .......... .......... .......... .......... ..........  249K
    50K .......... .......... .......... .......... ..........  562K
   100K .......... .......... .......... .......... .......... 4.50M
   150K .......... .......... .......... .......... ..........  562K
   200K ........

Archive:  master.zip
84925e34f619e4a5f8e92219d12655a0af1dd413
   creating: trappeua-master/
  inflating: trappeua-master/README.md  
  inflating: trappeua-master/ethanol.gro  
  inflating: trappeua-master/tip4p.gro  
   creating: trappeua-master/trappeua.ff/
  inflating: trappeua-master/trappeua.ff/alcohols.rtp  
  inflating: trappeua-master/trappeua.ff/alkanes.rtp  
  inflating: trappeua-master/trappeua.ff/aromatic.rtp  
  inflating: trappeua-master/trappeua.ff/atomtypes.atp  
  inflating: trappeua-master/trappeua.ff/ethanol.itp  
  inflating: trappeua-master/trappeua.ff/ffbonded.itp  
  inflating: trappeua-master/trappeua.ff/ffnonbonded.itp  
  inflating: trappeua-master/trappeua.ff/forcefield.doc  
  inflating: trappeua-master/trappeua.ff/forcefield.itp  
   creating: trappeua-master/trappeua.ff/hydrocarbons/
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C100H202.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C100H202.itp  
  inflating: trappeua-master/trappeua.

  inflating: trappeua-master/trappeua.ff/hydrocarbons/C43H88.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C43H88.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C43H88.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C44H90.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C44H90.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C44H90.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C45H92.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C45H92.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C45H92.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C46H94.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C46H94.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C46H94.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C47H96.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C47H96.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C47H96.p

  inflating: trappeua-master/trappeua.ff/hydrocarbons/C77H156.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C77H156.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C78H158.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C78H158.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C78H158.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C79H160.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C79H160.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C79H160.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C7H16.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C7H16.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C7H16.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C80H162.gro  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C80H162.itp  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/C80H162.pdb  
  inflating: trappeua-master/trappeua.ff/hydrocarbons/

Для того, чтобы GROMACS обнаружил поле и его можно было бы использовать, необходимо чтобы папка с расширением 'ff' находилась в рабочей директории. Поэтому, мы переместили данную папку после распаковки архива. Можно также получить силовое поле напрямую из репозитория:

In [ ]:
%%bash

git clone https://github.com/zhmurov/trappeua
mv trappeua/trappeua.ff .
rm -rf trappeua

Данная версия силового поля уже содержит описание и координаты для молекулы 5CB, которые находятся в папке 'trappeua.ff/liquidcrystals/', файл координат:

In [ ]:
!cat trappeua.ff/liquidcrystals/5CB.pdb

Файл топологии:

In [ ]:
!cat trappeua.ff/liquidcrystals/5CB.itp

Система будет включать 100 молекул внутрии куба с 4х4х4 нм. Для упаковки можно использовать программу Packmol.

In [ ]:
%%bash

git clone https://github.com/m3g/packmol.git
cd packmol
git checkout v20.3.5
./configure
make

Конфигурационный файл программы Packmol выглядит слудующим образом:

In [ ]:
!cat ../input/packmol.inp

Выполнив комманду:

In [ ]:
!packmol/packmol < ../input/packmol.inp

В результате получаем координаты:

In [ ]:
!cat conf.pdb

In [ ]:
Топология системы:

In [ ]:
!cp ../input/topol.top .
!cat topol.top

## Подготовка системы



In [ ]:
!gmx editconf -f conf.pdb -o conf.gro -box 4 4 4
!gmx grompp -f ../input/em.mdp -c conf.gro -o em.tpr
!gmx mdrun -deffnm em -nt 4

In [ ]:
!gmx grompp -f ../input/nvt.mdp -c em.gro -o nvt.tpr
!gmx mdrun -deffnm nvt -nt 4

In [ ]:
!gmx grompp -f ../input/npt.mdp -c nvt.gro -o npt.tpr
!gmx mdrun -deffnm npt -nt 4

## Моделирование

In [ ]:
%%bash

ELECTRIC_FIELD=1.0
cp ../input/md_iso_E.mdp md_iso_${ELECTRIC_FIELD}.mdp
sed -i "s/ELECTRIC_FIELD/${ELECTRIC_FIELD}/g" md_iso_${ELECTRIC_FIELD}.mdp

In [ ]:
!gmx grompp -f md_iso_${ELECTRIC_FIELD}.mdp -c npt.gro -o md_iso_${ELECTRIC_FIELD}.tpr
!gmx mdrun -deffnm md_iso_${ELECTRIC_FIELD} -nt 4

# Немного анализа результатов

In [ ]:
import MDAnalysis as mda

#electric_fields = ["0.00", "0.01", "0.1", "1.0", "10.0", "50.0"]
electric_fields = ["0.2", "0.5"]

for electric_field in electric_fields:

    u = mda.Universe("data/long/conf.pdb", "data/long/md_iso_" + electric_field + ".xtc", in_memory="True")
    step = 0
    order_params = []
    times  = []
    for ts in u.trajectory:
        box = ts.dimensions
        cosxi2 = 0.0
        ncosxi2 = 0
        ends1 = u.select_atoms("name N1")
        ends2 = u.select_atoms("name C19")
        for i in range(0, len(ends1) - 1):
            endi1 = ends1[i].position
            endi2 = ends2[i].position
            ri = endi2 - endi1
            ri[0] = ri[0] - round(ri[0] / box[0]) * box[0]
            ri[1] = ri[1] - round(ri[1] / box[1]) * box[1]
            ri[2] = ri[2] - round(ri[2] / box[2]) * box[2]
            ri_len = (ri[0]*ri[0] + ri[1]*ri[1] + ri[2]*ri[2]) ** 0.5
            for j in range(i + 1, len(ends1)):
                endj1 = ends1[j].position
                endj2 = ends2[j].position
                rj = endj2 - endj1
                rj[0] = rj[0] - round(rj[0] / box[0]) * box[0]
                rj[1] = rj[1] - round(rj[1] / box[1]) * box[1]
                rj[2] = rj[2] - round(rj[2] / box[2]) * box[2]
                rj_len = (rj[0]*rj[0] + rj[1]*rj[1] + rj[2]*rj[2]) ** 0.5
                cosxi = (ri[0]*rj[0] + ri[1]*rj[1] + ri[2]*rj[2]) / (ri_len * rj_len)
                cosxi2 += cosxi*cosxi
                ncosxi2 += 1
        cosxi2 /= ncosxi2
        order_param = (3.0*cosxi2 - 1.0)/2.0
        order_params.append(order_param)
        times.append(step)
        step += 1
        print(step, order_param)

    import matplotlib.pyplot as plt
    fig = plt.figure()
    plt.plot(times, order_params)
    plt.show()
    fig.savefig("order_parameter_" + electric_field + ".png")